In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Connect to data folder of NerLTR-DTA
root_path = '/content/gdrive/My Drive/KI JKU/Seminar_PracticalW_MasterThesis/MasterThesis/NerLTR/data/'  
import os
os.chdir(root_path)
!pwd

/content/gdrive/My Drive/KI JKU/Seminar_PracticalW_MasterThesis/MasterThesis/NerLTR/data


In [ ]:
import numpy as np
import pandas as pd
import linecache
import math
from collections import Counter
import pickle

In [ ]:
#Share
share_drug = [[5,1,4,2,5],
              [1,2,2,2,1],
              [4,2,5,2,4],
              [2,2,2,3,2],
              [5,1,4,2,5]]

share_target = [[3,1,2,3,3,3],
                [1,3,1,3,1,1],
                [2,1,3,3,2,2],
                [3,3,3,5,3,3],
                [3,1,2,3,3,3],
                [3,1,2,3,3,3]
                ]

In [ ]:
def check_symmetric(a, tol=1e-8):
    return np.all(np.abs(a-a.T) < tol)
#similarity
sim_drug = [[1,0.2,0.8,0.5,0.6],
            [0.2,1,0.9,0.1,0.5],
            [0.8,0.9,1,0.3,0.1],
            [0.5,0.1,0.3,1,0.9],
            [0.6,0.5,0.1,0.9,1]
            ]

sim_target = [[1,0.2,0.8,0.5,0.6,0.8],
              [0.2,1,0.9,0.1,0.5,0.3],
              [0.8,0.9,1,0.3,0.1,0.6],
              [0.5,0.1,0.3,1,0.9,0.9],
              [0.6,0.5,0.1,0.9,1,0.1],
              [0.8,0.3,0.6,0.9,0.1,1]
              ]

print(f'Sim Drug is sym: {check_symmetric(np.mat(sim_drug))}')
print(f'Sim Target is sym: {check_symmetric(np.mat(sim_target))}')

Sim Drug is sym: True
Sim Target is sym: True


In [ ]:
#Affinity
affi = [[10,np.nan,8,5,7,13],
        [np.nan,8,np.nan,6,np.nan,np.nan],
        [12,8,np.nan,9,11,13],
        [np.nan,13,8,11,np.nan,np.nan],
        [14,np.nan,8,9,11,9],
        ]

In [ ]:
#SSAF
ssaf_drug = []
for i in range(len(sim_drug)):
  ssaf_drug.append([np.mean(sim_drug[i]),
              np.median(sim_drug[i]),
              np.percentile(sim_drug[i], 75),
              np.percentile(sim_drug[i], 85),
              np.percentile(sim_drug[i], 95)
              ])
print(f'SSAF: {ssaf_drug}')
print(f'Len SSAF: {len(ssaf_drug[0])}')

ssaf_target = []
for i in range(len(sim_target)):
  ssaf_target.append([np.mean(sim_target[i]),
              np.median(sim_target[i]),
              np.percentile(sim_target[i], 75),
              np.percentile(sim_target[i], 85),
              np.percentile(sim_target[i], 95)
              ])
print(f'SSAF: {ssaf_target}')
print(f'Len SSAF: {len(ssaf_target[0])}')

SSAF: [[0.62, 0.6, 0.8, 0.88, 0.96], [0.54, 0.5, 0.9, 0.94, 0.98], [0.62, 0.8, 0.9, 0.94, 0.98], [0.5599999999999999, 0.5, 0.9, 0.94, 0.98], [0.62, 0.6, 0.9, 0.94, 0.98]]
Len SSAF: 5
SSAF: [[0.65, 0.7, 0.8, 0.8500000000000001, 0.95], [0.5, 0.4, 0.8, 0.925, 0.975], [0.6166666666666667, 0.7, 0.875, 0.925, 0.975], [0.6166666666666666, 0.7, 0.9, 0.925, 0.975], [0.5333333333333333, 0.55, 0.825, 0.925, 0.975], [0.6166666666666667, 0.7, 0.875, 0.925, 0.975]]
Len SSAF: 5


In [ ]:
def compute_mode(sample):
  c = Counter(sample)
  return [k for k, v in c.items() if v == c.most_common(1)[0][1]]
asaf_drugs=[]
print('Drugs')
for i in range(5):
  #ASAF --> Afffinity
  mean1 = np.mean(np.nan_to_num(affi[i]))   # compute mean including nan (full length of affinity e.g. KIBA=229)

  #removing all nan from array
  no_nan_array = np.array(affi[i])[~np.isnan(affi[i])]
  mean2 = np.mean(no_nan_array)  # compute mean excluding nan (all nan values ignored)
  number = len(no_nan_array)  #number of non-nan values
  mode = np.mean(compute_mode(no_nan_array))  #compute mean of all modes
  quantile_25 = np.percentile(no_nan_array, 25) #compute 25th quantile
  quantile_50 = np.percentile(no_nan_array, 50)  #compute 50th quantile
  quantile_75 = np.percentile(no_nan_array, 75)  #compute 75th quantile

  #Sort arary for lowest and highest values
  sorted_index_array = np.argsort(no_nan_array)# sorted array
  sorted_array = no_nan_array[sorted_index_array]  # sorted array from lowest to highest
  n=5 #n highest/lowest values
  asaf_5_high = sorted_array[-n:]
  asaf_5_low = sorted_array[:n]
  asaf=[mean1,mean2,number,mode,quantile_25,quantile_50,quantile_75,asaf_5_high,asaf_5_low]
  asaf_drugs.append(asaf)
  print(f'{i}:\tSSAF: {asaf_drugs[i]}')
print(f'Len SSAF: {len(asaf_drugs[0])}')

Drugs
0:	SSAF: [7.166666666666667, 8.6, 5, 8.6, 7.0, 8.0, 10.0, array([ 5.,  7.,  8., 10., 13.]), array([ 5.,  7.,  8., 10., 13.])]
1:	SSAF: [2.3333333333333335, 7.0, 2, 7.0, 6.5, 7.0, 7.5, array([6., 8.]), array([6., 8.])]
2:	SSAF: [8.833333333333334, 10.6, 5, 10.6, 9.0, 11.0, 12.0, array([ 8.,  9., 11., 12., 13.]), array([ 8.,  9., 11., 12., 13.])]
3:	SSAF: [5.333333333333333, 10.666666666666666, 3, 10.666666666666666, 9.5, 11.0, 12.0, array([ 8., 11., 13.]), array([ 8., 11., 13.])]
4:	SSAF: [8.5, 10.2, 5, 9.0, 9.0, 9.0, 11.0, array([ 8.,  9.,  9., 11., 14.]), array([ 8.,  9.,  9., 11., 14.])]
Len SSAF: 9


In [ ]:
asaf_targets=[]
print('Targets')
for i in range(6):
  aff_T = np.mat(affi).T
  #ASAF --> Afffinity
  mean1 = np.mean(np.nan_to_num(aff_T[i]))   # compute mean including nan (full length of affinity e.g. KIBA=229)

  #removing all nan from array
  no_nan_array = np.array(aff_T[i])[~np.isnan(aff_T[i])]
  mean2 = np.mean(no_nan_array)  # compute mean excluding nan (all nan values ignored)
  number = len(no_nan_array)  #number of non-nan values
  mode = np.mean(compute_mode(no_nan_array))  #compute mean of all modes
  quantile_25 = np.percentile(no_nan_array, 25) #compute 25th quantile
  quantile_50 = np.percentile(no_nan_array, 50)  #compute 50th quantile
  quantile_75 = np.percentile(no_nan_array, 75)  #compute 75th quantile

  #Sort arary for lowest and highest values
  sorted_index_array = np.argsort(no_nan_array)# sorted array
  sorted_array = no_nan_array[sorted_index_array]  # sorted array from lowest to highest
  n=5 #n highest/lowest values
  print(sorted_array)
  asaf_5_high = sorted_array[-n:]
  asaf_5_low = sorted_array[:n]
  asaf=[mean1,mean2,number,mode,quantile_25,quantile_50,quantile_75,asaf_5_high,asaf_5_low]
  asaf_targets.append(asaf)
  print(f'{i}:\tASAF: {asaf_targets[i]}')
print(f'Len ASAF: {len(asaf_targets[0])}')

Targets
[10. 12. 14.]
0:	ASAF: [7.2, 12.0, 3, 12.0, 11.0, 12.0, 13.0, array([10., 12., 14.]), array([10., 12., 14.])]
[ 8.  8. 13.]
1:	ASAF: [5.8, 9.666666666666666, 3, 8.0, 8.0, 8.0, 10.5, array([ 8.,  8., 13.]), array([ 8.,  8., 13.])]
[8. 8. 8.]
2:	ASAF: [4.8, 8.0, 3, 8.0, 8.0, 8.0, 8.0, array([8., 8., 8.]), array([8., 8., 8.])]
[ 5.  6.  9.  9. 11.]
3:	ASAF: [8.0, 8.0, 5, 9.0, 6.0, 9.0, 9.0, array([ 5.,  6.,  9.,  9., 11.]), array([ 5.,  6.,  9.,  9., 11.])]
[ 7. 11. 11.]
4:	ASAF: [5.8, 9.666666666666666, 3, 11.0, 9.0, 11.0, 11.0, array([ 7., 11., 11.]), array([ 7., 11., 11.])]
[ 9. 13. 13.]
5:	ASAF: [7.0, 11.666666666666666, 3, 13.0, 11.0, 13.0, 13.0, array([ 9., 13., 13.]), array([ 9., 13., 13.])]
Len ASAF: 9


In [ ]:
#GAAF Thresholds --> Index
def compute_index_threshold_1(sim_matrix, ssaf_feature):
  #loop through each row of the similarity matrix
  gaaf_t1_index=[]
  
  for i,row in enumerate(sim_matrix):
    threshold_mean = ssaf_feature[i][0]
    threshold_median = ssaf_feature[i][1]
    threshold_75 = ssaf_feature[i][2]
    threshold_85 = ssaf_feature[i][3]
    threshold_95 = ssaf_feature[i][4]

    #get index that have value higher than the thresholds
    index_mean = [idx for idx, val in enumerate(row) if val > threshold_mean]
    index_median = [idx for idx, val in enumerate(row) if val > threshold_median] 
    index_75 = [idx for idx, val in enumerate(row) if val > threshold_75] 
    index_85 = [idx for idx, val in enumerate(row) if val > threshold_85]
    index_95 = [idx for idx, val in enumerate(row) if val > threshold_95]

    #combine the index to one list 
    gaaf_t1_index.append(index_mean+index_median+index_75+index_85+index_95)

  return gaaf_t1_index

def compute_index_threshold_2(share_matrix):
  #loop through each row of the share matrix
  gaaf_t2_index=[]
  for row in share_matrix:
    threshold_75=np.percentile(row, 75)  #compute 75th quantile
    threshold_85=np.percentile(row, 85)  #compute 85th quantile
    threshold_95=np.percentile(row, 95)  #compute 95th quantile
    #get index that have value higher than the thresholds
    index_75 = [idx for idx, val in enumerate(row) if val > threshold_75] 
    index_85 = [idx for idx, val in enumerate(row) if val > threshold_85]
    index_95 = [idx for idx, val in enumerate(row) if val > threshold_95]
    #combine the index to one list 
    gaaf_t2_index.append(index_75+index_85+index_95)
  return gaaf_t2_index

thresh_1_drug = compute_index_threshold_1(sim_drug,ssaf_drug)
thresh_2_drug = compute_index_threshold_2(share_drug)

thresh_1_target = compute_index_threshold_1(sim_target,ssaf_target)
thresh_2_target = compute_index_threshold_2(share_target)

In [ ]:
thresh_1_drug

[[0, 2, 0, 2, 0, 0, 0],
 [1, 2, 1, 2, 1, 1, 1],
 [0, 1, 2, 1, 2, 2, 2, 2],
 [3, 4, 3, 4, 3, 3, 3],
 [3, 4, 3, 4, 4, 4, 4]]

In [ ]:
thresh_2_drug

[[], [], [2, 2, 2], [3, 3, 3], []]

In [ ]:
#GAAF 
def gen_gaaf(thresh1, thresh2, asaf): 

  thresh=thresh1+thresh2
  thresh = [subitem for item in thresh for subitem in item]
  gaaf_full=[]
  for i, el in enumerate(thresh1):
    thresh=el+thresh2[i]
    gaaf=[]
    for idx in thresh:
      gaaf.append(asaf[idx][0])
      gaaf.append(asaf[idx][1])
      gaaf.append(asaf[idx][2])
      gaaf.append(asaf[idx][3])
      gaaf.append(asaf[idx][4])
      gaaf.append(asaf[idx][5])
      gaaf.append(asaf[idx][6])
      for el in asaf[idx][7]:
        gaaf.append(el)
      for el in asaf[idx][8]:
        gaaf.append(el)
    g = [np.min(gaaf),
        np.max(gaaf),
        np.percentile(gaaf, 75),
        np.percentile(gaaf, 50),
        np.percentile(gaaf, 25),
        np.mean(gaaf),
        np.mean(compute_mode(gaaf))]
    gaaf_full.append(g)
  return gaaf_full

gaaf_drugs=gen_gaaf(thresh_1_drug,thresh_2_drug,asaf_drugs)
gaaf_targets=gen_gaaf(thresh_1_target,thresh_2_target,asaf_targets)
print(f'Len GAAF: {len(gaaf_drugs[0])}')
gaaf_drugs[0]

Len GAAF: 7


[5.0, 13.0, 10.3, 8.6, 7.0, 8.805882352941177, 8.0]

In [ ]:
gaaf_targets[0]

[3.0, 14.0, 12.0, 11.0, 8.0, 10.136182336182337, 12.0]

In [ ]:
#AAAF Index
def AAAF_indexes(sim):
  #Sort arary for highes and lowest values
  top_5_similar_index=[]
  n=5 #n most similar index
  for s in sim: 
    print(s)
    top_5_similar_index.append(np.argsort(-np.array(s))[1:n+1].tolist()) #sort and get 5 most similar
  print(top_5_similar_index)
  #get 5 similar for each of the 5 most similar 
  aaaf_index=[]
  for row in top_5_similar_index:
    all_25_index=[]
    all_25_index.append([top_5_similar_index[i] for i in row])
    flat_list = [subitem for item in all_25_index for subitem in item]
    flat_list = [subitem for item in flat_list for subitem in item]
    aaaf_index.append(flat_list)
  return aaaf_index

aaaf_index_drugs=AAAF_indexes(sim_drug)
aaaf_index_targets=AAAF_indexes(sim_target)

[1, 0.2, 0.8, 0.5, 0.6]
[0.2, 1, 0.9, 0.1, 0.5]
[0.8, 0.9, 1, 0.3, 0.1]
[0.5, 0.1, 0.3, 1, 0.9]
[0.6, 0.5, 0.1, 0.9, 1]
[[0, 2, 4, 3, 1], [1, 2, 4, 0, 3], [2, 1, 0, 3, 4], [3, 4, 0, 2, 1], [4, 3, 0, 1, 2]]
[1, 0.2, 0.8, 0.5, 0.6, 0.8]
[0.2, 1, 0.9, 0.1, 0.5, 0.3]
[0.8, 0.9, 1, 0.3, 0.1, 0.6]
[0.5, 0.1, 0.3, 1, 0.9, 0.9]
[0.6, 0.5, 0.1, 0.9, 1, 0.1]
[0.8, 0.3, 0.6, 0.9, 0.1, 1]
[[0, 2, 5, 4, 3], [1, 2, 4, 5, 0], [2, 1, 0, 5, 3], [3, 4, 5, 0, 2], [4, 3, 0, 1, 2], [5, 3, 0, 2, 1]]


In [ ]:
#AAAF
def generate_AAAF_feature(index_file, asaf):
  aaaf_features=[]
  for row in index_file: #loop through each row of index file --> contains 25 index
    asaf_features = []
    print(row)
    for idx in row:
        asaf_features.append(asaf[idx][0])
        asaf_features.append(asaf[idx][1])
        asaf_features.append(asaf[idx][2])
        asaf_features.append(asaf[idx][3])
        asaf_features.append(asaf[idx][4])
        asaf_features.append(asaf[idx][5])
        asaf_features.append(asaf[idx][6])
        for el in asaf[idx][7]:
          asaf_features.append(el)
        for el in asaf[idx][8]:
          asaf_features.append(el)  
    aaaf_features.append(asaf_features)
  return aaaf_features

aaaf_drugs=generate_AAAF_feature(aaaf_index_drugs,asaf_drugs)
aaaf_targets=generate_AAAF_feature(aaaf_index_targets,asaf_targets)
print(len(aaaf_drugs[0]))
aaaf_drugs[0]

[0, 2, 4, 3, 1, 2, 1, 0, 3, 4, 4, 3, 0, 1, 2, 3, 4, 0, 2, 1, 1, 2, 4, 0, 3]
[1, 2, 4, 0, 3, 2, 1, 0, 3, 4, 4, 3, 0, 1, 2, 0, 2, 4, 3, 1, 3, 4, 0, 2, 1]
[2, 1, 0, 3, 4, 1, 2, 4, 0, 3, 0, 2, 4, 3, 1, 3, 4, 0, 2, 1, 4, 3, 0, 1, 2]
[3, 4, 0, 2, 1, 4, 3, 0, 1, 2, 0, 2, 4, 3, 1, 2, 1, 0, 3, 4, 1, 2, 4, 0, 3]
[4, 3, 0, 1, 2, 3, 4, 0, 2, 1, 0, 2, 4, 3, 1, 1, 2, 4, 0, 3, 2, 1, 0, 3, 4]
[0, 2, 5, 4, 3, 2, 1, 0, 5, 3, 5, 3, 0, 2, 1, 4, 3, 0, 1, 2, 3, 4, 5, 0, 2]
[1, 2, 4, 5, 0, 2, 1, 0, 5, 3, 4, 3, 0, 1, 2, 5, 3, 0, 2, 1, 0, 2, 5, 4, 3]
[2, 1, 0, 5, 3, 1, 2, 4, 5, 0, 0, 2, 5, 4, 3, 5, 3, 0, 2, 1, 3, 4, 5, 0, 2]
[3, 4, 5, 0, 2, 4, 3, 0, 1, 2, 5, 3, 0, 2, 1, 0, 2, 5, 4, 3, 2, 1, 0, 5, 3]
[4, 3, 0, 1, 2, 3, 4, 5, 0, 2, 0, 2, 5, 4, 3, 1, 2, 4, 5, 0, 2, 1, 0, 5, 3]
[5, 3, 0, 2, 1, 3, 4, 5, 0, 2, 0, 2, 5, 4, 3, 2, 1, 0, 5, 3, 1, 2, 4, 5, 0]
375


[7.166666666666667,
 8.6,
 5,
 8.6,
 7.0,
 8.0,
 10.0,
 5.0,
 7.0,
 8.0,
 10.0,
 13.0,
 5.0,
 7.0,
 8.0,
 10.0,
 13.0,
 8.833333333333334,
 10.6,
 5,
 10.6,
 9.0,
 11.0,
 12.0,
 8.0,
 9.0,
 11.0,
 12.0,
 13.0,
 8.0,
 9.0,
 11.0,
 12.0,
 13.0,
 8.5,
 10.2,
 5,
 9.0,
 9.0,
 9.0,
 11.0,
 8.0,
 9.0,
 9.0,
 11.0,
 14.0,
 8.0,
 9.0,
 9.0,
 11.0,
 14.0,
 5.333333333333333,
 10.666666666666666,
 3,
 10.666666666666666,
 9.5,
 11.0,
 12.0,
 8.0,
 11.0,
 13.0,
 8.0,
 11.0,
 13.0,
 2.3333333333333335,
 7.0,
 2,
 7.0,
 6.5,
 7.0,
 7.5,
 6.0,
 8.0,
 6.0,
 8.0,
 8.833333333333334,
 10.6,
 5,
 10.6,
 9.0,
 11.0,
 12.0,
 8.0,
 9.0,
 11.0,
 12.0,
 13.0,
 8.0,
 9.0,
 11.0,
 12.0,
 13.0,
 2.3333333333333335,
 7.0,
 2,
 7.0,
 6.5,
 7.0,
 7.5,
 6.0,
 8.0,
 6.0,
 8.0,
 7.166666666666667,
 8.6,
 5,
 8.6,
 7.0,
 8.0,
 10.0,
 5.0,
 7.0,
 8.0,
 10.0,
 13.0,
 5.0,
 7.0,
 8.0,
 10.0,
 13.0,
 5.333333333333333,
 10.666666666666666,
 3,
 10.666666666666666,
 9.5,
 11.0,
 12.0,
 8.0,
 11.0,
 13.0,
 8.0,
 11.0,
 13.0

In [ ]:
print(f'Len SSAF: {len(ssaf_drug[0])}')
print(f'Len ASAF: {len(asaf_drugs[0])}')
print(f'Len GAAF: {len(gaaf_drugs[0])}')
print(f'Len ASAF: {len(aaaf_drugs[0])}')
print(f'Len Combined: {len(ssaf_drug[0])+len(asaf_drugs[0])+len(gaaf_drugs[0])+len(aaaf_drugs[0])}')

Len SSAF: 5
Len ASAF: 9
Len GAAF: 7
Len ASAF: 232
Len Combined: 253


In [ ]:
full = ssaf_drug[0]

full.append(asaf_drugs[0][0])
full.append(asaf_drugs[0][1])
full.append(asaf_drugs[0][2])
full.append(asaf_drugs[0][3])
full.append(asaf_drugs[0][4])
full.append(asaf_drugs[0][5])
full.append(asaf_drugs[0][6])
for el in asaf_drugs[0][7]:
  full.append(el)
for el in asaf_drugs[0][8]:
  full.append(el)

full = full +gaaf_drugs[0]
full = full +aaaf_drugs[0]

len(full)

261

In [ ]:
np.load('test/share_drugs.npy')

array([[5., 1., 4., 2., 5.],
       [1., 2., 2., 2., 1.],
       [4., 2., 5., 2., 4.],
       [2., 2., 2., 3., 2.],
       [5., 1., 4., 2., 5.]])

In [ ]:
np.load('test/share_targets.npy')

array([[3., 1., 2., 3., 3., 3.],
       [1., 3., 1., 3., 1., 1.],
       [2., 1., 3., 3., 2., 2.],
       [3., 3., 3., 5., 3., 3.],
       [3., 1., 2., 3., 3., 3.],
       [3., 1., 2., 3., 3., 3.]])

In [ ]:
path_test_SSAF_drugs = 'test/SAF/SSAF_drugs' #SSAF feature of drug
path_test_SSAF_targets = 'test/SAF/SSAF_targets' #SSAF feature of target
path_test_ASAF_drugs = 'test/SAF/ASAF_drugs' #ASAF feature of drug
path_test_ASAF_targets = 'test/SAF/ASAF_targets' #ASAF feature of target
path_test_GAAF_drugs = 'test/AAF/GAAF_drugs' #GAAF feature of drugs
path_test_GAAF_targets = 'test/AAF/GAAF_targets' #GAAF feature of target
path_test_AAAF_drugs = 'test/AAF/AAAF_drugs' #AAAF feature of drugs
path_test_AAAF_targets = 'test/AAF/AAAF_targets' #AAAF feature of target

In [ ]:
with open(path_test_SSAF_targets, "rb") as fp:   # Unpickling SSAF features targets
    ssaf_targets_test = pickle.load(fp)
with open(path_test_ASAF_targets, "rb") as fp:   # Unpickling ASAF features targets
    asaf_targets_test = pickle.load(fp)
with open(path_test_GAAF_targets, "rb") as fp:   # Unpickling GAAF features targets
    gaaf_targets_test = pickle.load(fp)
with open(path_test_AAAF_targets, "rb") as fp:   # Unpickling AAAF features targets
    aaaf_targets_test = pickle.load(fp)

In [ ]:
print(ssaf_targets_test)

       mean  median    75q    85q    95q
0  0.650000    0.70  0.800  0.850  0.950
1  0.500000    0.40  0.800  0.925  0.975
2  0.616667    0.70  0.875  0.925  0.975
3  0.616667    0.70  0.900  0.925  0.975
4  0.533333    0.55  0.825  0.925  0.975
5  0.616667    0.70  0.875  0.925  0.975


In [ ]:
print(asaf_targets_test)

   mean_1     mean_2 number  mode   25q   50q   75q  \
0     7.2  12.000000      3  12.0  11.0  12.0  13.0   
1     5.8   9.666667      3   8.0   8.0   8.0  10.5   
2     4.8   8.000000      3   8.0   8.0   8.0   8.0   
3     8.0   8.000000      5   9.0   6.0   9.0   9.0   
4     5.8   9.666667      3  11.0   9.0  11.0  11.0   
5     7.0  11.666667      3  13.0  11.0  13.0  13.0   

                       5_high                       5_low  
0          [10.0, 12.0, 14.0]          [10.0, 12.0, 14.0]  
1            [8.0, 8.0, 13.0]            [8.0, 8.0, 13.0]  
2             [8.0, 8.0, 8.0]             [8.0, 8.0, 8.0]  
3  [5.0, 6.0, 9.0, 9.0, 11.0]  [5.0, 6.0, 9.0, 9.0, 11.0]  
4           [7.0, 11.0, 11.0]           [7.0, 11.0, 11.0]  
5           [9.0, 13.0, 13.0]           [9.0, 13.0, 13.0]  


In [ ]:
print(gaaf_targets_test)

   min   max  upper_q  median  lower_q       mean  mode
0  3.0  14.0     12.0    11.0      8.0  10.136182  12.0
1  3.0  13.0      9.0     8.0      8.0   8.201814   8.0
2  3.0  14.0      9.0     8.0      8.0   8.288125   8.0
3  3.0  13.0     11.0     9.0      6.0   8.469504   9.0
4  3.0  14.0     11.0     9.0      7.0   9.005164  11.0
5  3.0  14.0     13.0    10.0      8.0   9.822066  13.0


In [ ]:
print(aaaf_targets_test[0])
len(aaaf_targets_test[0])

[7.2, 12.0, 3, 12.0, 11.0, 12.0, 13.0, 10.0, 12.0, 14.0, 14.0, 14.0, 14.0, 4.8, 8.0, 3, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 7.0, 11.666666666666666, 3, 13.0, 11.0, 13.0, 13.0, 9.0, 13.0, 13.0, 13.0, 13.0, 13.0, 5.8, 9.666666666666666, 3, 11.0, 9.0, 11.0, 11.0, 7.0, 11.0, 11.0, 11.0, 11.0, 11.0, 8.0, 8.0, 5, 9.0, 6.0, 9.0, 9.0, 5.0, 6.0, 9.0, 9.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 4.8, 8.0, 3, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 5.8, 9.666666666666666, 3, 8.0, 8.0, 8.0, 10.5, 8.0, 8.0, 13.0, 13.0, 13.0, 13.0, 7.2, 12.0, 3, 12.0, 11.0, 12.0, 13.0, 10.0, 12.0, 14.0, 14.0, 14.0, 14.0, 7.0, 11.666666666666666, 3, 13.0, 11.0, 13.0, 13.0, 9.0, 13.0, 13.0, 13.0, 13.0, 13.0, 8.0, 8.0, 5, 9.0, 6.0, 9.0, 9.0, 5.0, 6.0, 9.0, 9.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 7.0, 11.666666666666666, 3, 13.0, 11.0, 13.0, 13.0, 9.0, 13.0, 13.0, 13.0, 13.0, 13.0, 8.0, 8.0, 5, 9.0, 6.0, 9.0, 9.0, 5.0, 6.0, 9.0, 9.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 7.2, 12.0, 3, 12.0, 11.0, 12.0

345

In [ ]:
with open(path_test_SSAF_drugs, "rb") as fp:   # Unpickling SSAF features drugs
    ssaf_drugs_test = pickle.load(fp)
with open(path_test_ASAF_drugs, "rb") as fp:   # Unpickling ASAF features drugs
    asaf_drugs_test = pickle.load(fp)
with open(path_test_GAAF_drugs, "rb") as fp:   # Unpickling GAAF features drugs
    gaaf_drugs_test = pickle.load(fp)
with open(path_test_AAAF_drugs, "rb") as fp:   # Unpickling AAAF features drugs
    aaaf_drugs_test = pickle.load(fp)

In [ ]:
print(ssaf_drugs_test)

   mean  median  75q   85q   95q
0  0.62     0.6  0.8  0.88  0.96
1  0.54     0.5  0.9  0.94  0.98
2  0.62     0.8  0.9  0.94  0.98
3  0.56     0.5  0.9  0.94  0.98
4  0.62     0.6  0.9  0.94  0.98


In [ ]:
print(asaf_drugs_test)

     mean_1     mean_2 number       mode  25q   50q   75q  \
0  7.166667   8.600000      5   8.600000  7.0   8.0  10.0   
1  2.333333   7.000000      2   7.000000  6.5   7.0   7.5   
2  8.833333  10.600000      5  10.600000  9.0  11.0  12.0   
3  5.333333  10.666667      3  10.666667  9.5  11.0  12.0   
4  8.500000  10.200000      5   9.000000  9.0   9.0  11.0   

                         5_high                         5_low  
0   [5.0, 7.0, 8.0, 10.0, 13.0]   [5.0, 7.0, 8.0, 10.0, 13.0]  
1                    [6.0, 8.0]                    [6.0, 8.0]  
2  [8.0, 9.0, 11.0, 12.0, 13.0]  [8.0, 9.0, 11.0, 12.0, 13.0]  
3             [8.0, 11.0, 13.0]             [8.0, 11.0, 13.0]  
4   [8.0, 9.0, 9.0, 11.0, 14.0]   [8.0, 9.0, 9.0, 11.0, 14.0]  


In [ ]:
print(gaaf_drugs_test)

   min   max  upper_q     median  lower_q      mean       mode
0  5.0  13.0     10.3   8.600000     7.00  8.805882   8.000000
1  2.0  13.0      9.0   7.500000     6.50  7.671161   7.000000
2  2.0  13.0     11.0   9.000000     8.00  9.481714  10.666667
3  3.0  14.0     11.0  10.666667     8.00  9.686473  11.000000
4  3.0  14.0     11.0   9.000000     8.75  9.647147   9.000000


In [ ]:
print(aaaf_drugs_test[0])
len(aaaf_drugs_test[0])

[7.166666666666667, 8.6, 5, 8.6, 7.0, 8.0, 10.0, 5.0, 7.0, 8.0, 10.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 8.833333333333334, 10.6, 5, 10.6, 9.0, 11.0, 12.0, 8.0, 9.0, 11.0, 12.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 8.5, 10.2, 5, 9.0, 9.0, 9.0, 11.0, 8.0, 9.0, 9.0, 11.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 5.333333333333333, 10.666666666666666, 3, 10.666666666666666, 9.5, 11.0, 12.0, 8.0, 11.0, 13.0, 13.0, 13.0, 13.0, 2.3333333333333335, 7.0, 2, 7.0, 6.5, 7.0, 7.5, 6.0, 8.0, 8.0, 8.0, 8.833333333333334, 10.6, 5, 10.6, 9.0, 11.0, 12.0, 8.0, 9.0, 11.0, 12.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 2.3333333333333335, 7.0, 2, 7.0, 6.5, 7.0, 7.5, 6.0, 8.0, 8.0, 8.0, 7.166666666666667, 8.6, 5, 8.6, 7.0, 8.0, 10.0, 5.0, 7.0, 8.0, 10.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 5.333333333333333, 10.666666666666666, 3, 10.666666666666666, 9.5, 11.0, 12.0, 8.0, 11.0, 13.0, 13.0, 13.0, 13.0, 8.5, 10.2, 5, 9.0, 9.0, 9.0, 11.0, 8.0, 9.0, 9.0, 11.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 8.5, 10.2, 5, 9.0, 9.0,

375